In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os
import json

In [2]:
os.chdir('./distancevalidation1/')

f = open('alphapose-results.json')

data = json.load(f)
    
f.close()

In [3]:
## Groups same frames together

data_by_frames = []
curr_frame = 0
curr_kps = []

for d in data:
    if (d['image_id'] == (str(curr_frame) + '.jpg') ):
        curr_kps.append(d)
    else:
        
        data_by_frames.append(curr_kps.copy())
        curr_kps.clear()
        curr_kps.append(d)
        curr_frame += 1

In [4]:
cap = cv2.VideoCapture("AlphaPose_output.mp4")

ret, frame = cap.read()
pisteLines = {}
pisteLineIdentifiers = ['middleLines', 'leftEngardeLine', 'rightEngardeLine', 'leftWarningLine', 'rightWarningLine', 'leftEndLine', 'rightEndLine']
curr = 0
ptList = []

def onMousePisteLines(event, x, y, flags, param):
    global pisteLines
    global pisteLineIdentifiers
    global curr
    global ptList
    if event == cv2.EVENT_LBUTTONDOWN:
        ptList.append((x, y))
    elif event == cv2.EVENT_RBUTTONDOWN:
        print(ptList)
        avg_x = (ptList[0][0] + ptList[1][0]) / 2
        avg_y = (ptList[0][1] + ptList[1][1]) / 2
        pisteLines[pisteLineIdentifiers[curr]] = (avg_x, avg_y)
        curr += 1
        ptList.clear()

cv2.imshow('WindowName', frame)
cv2.setMouseCallback('WindowName', onMousePisteLines)

while(1):
    if cv2.waitKey(10) & 0xFF==ord('q'):
        cv2.destroyAllWindows()
        break
        
cap.release()
cv2.destroyAllWindows()

[(1087, 588), (1149, 716)]
[(480, 618), (451, 759)]
[(1620, 561), (1760, 678)]


In [5]:
data_by_frames[0]

[{'image_id': '0.jpg',
  'category_id': 1,
  'keypoints': [316.565185546875,
   177.12403869628906,
   0.952835202217102,
   316.565185546875,
   164.49612426757812,
   0.924065351486206,
   310.2512512207031,
   164.49612426757812,
   0.9693357348442078,
   266.0535583496094,
   164.49612426757812,
   0.7663102746009827,
   272.3675231933594,
   170.81008911132812,
   0.9365411996841431,
   196.60008239746094,
   215.00775146484375,
   0.9213711023330688,
   253.4256591796875,
   259.2054443359375,
   0.8488119840621948,
   139.77450561523438,
   297.0891418457031,
   0.9793621897697449,
   234.48379516601562,
   379.1705322265625,
   0.7928688526153564,
   196.60008239746094,
   347.60076904296875,
   0.8763632774353027,
   259.7396240234375,
   473.8798522949219,
   0.9225308895111084,
   183.97216796875,
   404.4263610839844,
   0.7753100991249084,
   240.79776000976562,
   423.36822509765625,
   0.699519157409668,
   190.2861328125,
   543.3333129882812,
   0.9207480549812317,
   

In [6]:
def average_keypoints(keypoints, confidence_threshold):
    total_considered = 0
    total_y = 0
    total_x = 0
    counter = 0
    kx = 0
    ky = 0
    for count, kp in enumerate(keypoints):
        if (count % 3 == 0):
            kx = kp
        elif (count % 3 == 1):
            ky = kp
        else:
            if kp > confidence_threshold:
                total_considered += 1
                total_y += ky
                total_x += kx
            kx = 0
            ky = 0
    
    average_y = total_y / total_considered
    average_x = total_x / total_considered
    
    return average_x, average_y

average_xy = []
count = 0
for i in data_by_frames:
    idstr = i[0]['image_id'].split('.')[0]
    while (count) < int(idstr):
        average_xy.append(average_xy[-1])
        count += 1
    avg0_x, avg0_y = average_keypoints(i[0]['keypoints'], 0.1)
    average_xy.append([avg0_x, avg0_y])
    count += 1

In [7]:
import math

def distCalc(pt1, pt2):
    dis = math.sqrt((pt2[0]-pt1[0])**2 + (pt2[1]-pt1[1])**2)
    return dis

def calc_dist_from_4m(fencer):
    # Negative if to the left of engarde, positive otherwise
    if (fencer[0] < pisteLines['leftEngardeLine'][0]):
        # Left side
        distFromLeftEGLine = leftEngardeToLeftWarning * (fencer[0] - pisteLines['leftEngardeLine'][0])
        return distFromLeftEGLine - 200
    elif (fencer[0] < pisteLines['middleLines'][0]):
        distFromMidLine = leftEnGardeToMid * (fencer[0] - pisteLines['middleLines'][0])
        return distFromMidLine
    elif (fencer[0] < pisteLines['rightEngardeLine'][0]):
        distFromMidLine = rightEnGardeToMid * (fencer[0] - pisteLines['middleLines'][0])
        return distFromMidLine
    else:
        distFromRightEGLine = rightEngardeToRightWarning * (fencer[0] - pisteLines['rightEngardeLine'][0])
        return 200 + distFromRightEGLine

In [8]:
leftEnGardeToMid = 200 / distCalc(pisteLines['middleLines'], pisteLines['leftEngardeLine'])
rightEnGardeToMid = 200 / distCalc(pisteLines['middleLines'], pisteLines['rightEngardeLine'])
leftEngardeToLeftWarning = leftEnGardeToMid
rightEngardeToRightWarning = rightEnGardeToMid

In [9]:
dist_from_4 = []
for coord in average_xy:
    dist_from_4.append(calc_dist_from_4m(coord))

## Add distance measurement caption to video

In [10]:
for val in pisteLines.values():
    print(val)

(1118.0, 652.0)
(465.5, 688.5)
(1690.0, 619.5)


In [11]:
import cv2
import numpy as np

cap = cv2.VideoCapture("AlphaPose_output.mp4")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4',fourcc, 30, (frame_width,frame_height))   
count = 0

while(True):
    ret, frame = cap.read()
    try:
        dist_str = str(dist_from_4[count]) + ' cm from mid'
        x, y = average_xy[count]
        x = math.floor(x)
        y = math.floor(y)
        coord = (x, y)
    except:
        dist_str = '0 cm from mid'
        x, y = (50, 50)
        coord = (x, y)
        
    if ret == True: 
        # Apply mask
        strings = ["0 cm", "-200 cm", "200 cm"]
        for iteration, val in enumerate(pisteLines.values()):
            x1, y1 = val
            x1 = int(x1)
            y1 = int(y1)
            frame = cv2.circle(frame, (x1,y1), 1, (0,0,255), 10)
            frame = cv2.putText(frame, strings[iteration], (x1,y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        frame = cv2.putText(frame, dist_str, coord, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        frame = cv2.line(frame, coord,(x, y + 400), (255,255,255), 2)
        
        # Write the frame into the file 'output.mp4'
        out.write(frame)
        
        cv2.imwrite(f"{count}output.jpg", frame)
        
        # Display the resulting frame    
        cv2.imshow('frame',frame)

        # Press Q on keyboard to stop recording
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        print(frame_width)
        break
    count += 1
 

out.release()
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

1920


In [12]:
count

782

In [13]:
len(average_xy)

781